In [1]:
import os 
import pandas as pd
from utils.data import *
from tqdm.auto import tqdm
tqdm.pandas()
#-------------
# globals
#-------------
src_dir                 =   os.path.join(os.getcwd(),"tests","ILMT_TAGSET_TEST")
model_path              =   os.path.join(src_dir,"keras_mlp_bangla.h5")
tagged_data_path        =   os.path.join(src_dir,"nlp_data_bn_tagged_mod.txt")
untaggeed_data_path     =   os.path.join(src_dir,"untagged.bn.txt")


# Data EDA

In [2]:
EDA=False
if EDA:
    #----------------
    # dataset EDA
    #---------------
    # get dataFrame
    df=textfile_to_dataset(tagged_data_path,eda=True)
    # get wtc
    words=[]
    tags=[]
    counts=[]
    for word in tqdm(df.word.unique()):
        words.append(word)
        _wdf=df.loc[df.word==word]
        tags.append(_wdf.tag.unique())
        counts.append(len(_wdf))
    # format and save dataframe
    dfs=pd.DataFrame({"word":words,"tags":tags,"count":counts})
    dfs = dfs.sort_values(by=['count'], ascending=False)
    dfs.to_csv(os.path.join(src_dir,"tagged_data_wtc.csv"),index=False)

#dfs


# Vectorizing Data and Encoding Labels

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
#----------------
# dataset 
#---------------
# get features and classes
X,y=textfile_to_dataset(tagged_data_path,eda=False)
# create dict vectorizer
dict_vectorizer = DictVectorizer(sparse=False)
dict_vectorizer.fit(X)
# create label encoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
print("Done!")

  0%|          | 0/2927 [00:00<?, ?it/s]

  0%|          | 0/2927 [00:00<?, ?it/s]

Done!


# Model Analysis

In [4]:
import keras
model=keras.models.load_model(model_path)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               24136192  
_________________________________________________________________
activation_1 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
activation_2 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 36)               

# Sample Test

In [5]:
from utils.tagger import BanglaPOSTagger
tagger=BanglaPOSTagger(model,label_encoder,dict_vectorizer)


[('দুশ্চিন্তার', 'NC'), ('কোন', 'JQ'), ('কারণই', 'NC'), ('নাই', 'CX')]

In [9]:
# test
sentence = 'দুশ্চিন্তার কোন কারণই নাই'
tagger.tag(sentence)

[('দুশ্চিন্তার', 'NC'), ('কোন', 'JQ'), ('কারণই', 'NC'), ('নাই', 'CX')]

# Untagged Data

In [8]:
# read the tagged_data
with open(text_file, encoding='utf8') as f:
    # read lines
    texts=f.readlines()


  0%|          | 0/1000 [00:00<?, ?it/s]

,word,tag
0,আবার,আবার
1,সবাইকে,সবাইকে
2,যার,যার
3,যার,যার
4,ইচ্ছামতো,ইচ্ছামতো
...,...,...
11303,ফসলের,ফসলের
11304,অর্ধেক,অর্ধেক
11305,জোতদারদের,জোতদারদের
11306,দিতে,দিতে


In [ ]:
for line in texts:
    